# Hyperfine solutions and spin-density for H2S

Compute the hyperfine energies and wavefunctions for selected rotational cluster states of $\text{H}_2\text{S}$.
The hyperfine interaction is represented by the spin-rotation coupling.

We also compute the nuclear spin densities to verify if the ortho-para interaction can induce the nuclear spin polarisation.

In [3]:
! export JAX_PLATFORMS=cpu

/gpfs/cfel/group/cmi/common/psi4/psi4conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [25]:
from typing import Dict, List

import h5py
import hdfdict
import jax
import numpy as np
from jax import config
from jax import numpy as jnp
from scipy import constants

from rovib.cartens import CART_IND
from rovib.spinrot_xy2 import spinrot_xy2
from rovib.symtop import threej_wang

config.update("jax_enable_x64", True)
KHZ_TO_INVCM = 1.0 / constants.value("speed of light in vacuum") * 10

In [26]:
with h5py.File("h2s_cluster_states_me.h5", "r") as h5:
    enr = hdfdict.load(h5.require_group("energies"), False)
    sr = hdfdict.load(h5.require_group("spin-rotation"), False)

enr = {
    int(J): {sym: enr_sym for sym, enr_sym in enr_J.items()} for J, enr_J in enr.items()
}

sr_h1 = {
    (int(J), int(J)): {
        (sym1, sym2): me_sym2
        for sym1, me_sym1 in me_J.items()
        for sym2, me_sym2 in me_sym1.items()
    }
    for J, me_J in sr["h1"].items()
}

sr_h2 = {
    (int(J), int(J)): {
        (sym1, sym2): me_sym2
        for sym1, me_sym1 in me_J.items()
        for sym2, me_sym2 in me_sym1.items()
    }
    for J, me_J in sr["h2"].items()
}

In [27]:
# sr_h1[(50,50)][('A1','A1')].shape
enr

{50: {'A1': array([24708.81890006]),
  'A2': array([24708.81879579]),
  'B1': array([24708.81890005]),
  'B2': array([24708.81879579])},
 51: {'A1': array([25447.36424769]),
  'A2': array([25447.36433043]),
  'B1': array([25447.36424769]),
  'B2': array([25447.36433042])},
 52: {'A1': array([26191.9451634]),
  'A2': array([26191.94505483]),
  'B1': array([26191.9451634]),
  'B2': array([26191.94505481])},
 53: {'A1': array([26942.10734119]),
  'A2': array([26942.11008082]),
  'B1': array([26942.10807142]),
  'B2': array([26942.11008082])},
 54: {'A1': array([27697.40016773]),
  'A2': array([27697.39981967]),
  'B1': array([27697.40016774]),
  'B2': array([27697.39981959])},
 55: {'A1': array([28457.34565511]),
  'A2': array([28457.34566424]),
  'B1': array([28457.34565512]),
  'B2': array([28457.34566424])},
 56: {'A1': array([29221.46994552]),
  'A2': array([29221.46994391]),
  'B1': array([29221.46994552]),
  'B2': array([29221.46994391])},
 57: {'A1': array([29989.28428709]),
  'A2'

In [32]:
F = 50
J = F

hyper_enr, hyper_vec, hyper_qua = spinrot_xy2(
    F,
    {J: enr[J]},
    {(J, J): sr_h1[(J, J)]},
    {(J, J): sr_h2[(J, J)]},
    spin_states=((0, "B2"), (1, "A1")),
    spins=(0.5, 0.5),
)

for sym in hyper_enr.keys():
    ind = np.argmax(hyper_vec[sym] ** 2, axis=0)
    for ii, (e, i) in enumerate(zip(hyper_enr[sym], ind)):
        qua = hyper_qua[sym][i]
        rov_sym = qua[1]
        print(
            sym,
            e,
            hyper_qua[sym][i],
            (e - enr[J][rov_sym]) / KHZ_TO_INVCM,
            hyper_vec[sym][i, ii],
        )

print(hyper_enr["B1"])
print(enr[J]["B1"], enr[J]["A2"])

# for key, val in hyper_enr.items():
#     print(key, val.shape, val)
# print(enr[J])

0 0 50 50 A2 A2 [[0.+0.j]]
0 1 50 50 A2 B1 [[-3.06568868e-05+0.j]]
1 0 50 50 B1 A2 [[-3.06568868e-05+0.j]]
1 1 50 50 B1 B1 [[-5.26985695e-07+0.j]]
0 0 50 50 A1 A1 [[0.+0.j]]
0 1 50 50 A1 B2 [[3.06568868e-05+0.j]]
1 0 50 50 B2 A1 [[3.06568868e-05+0.j]]
1 1 50 50 B2 B2 [[-5.26985669e-07+0.j]]
B1 24708.81878741142 (50, 'A2', 0, 'B2') [-251.31312801] (-0.9645884203858242-0j)
B1 24708.81890790879 (50, 'B1', 1, 'A1') [235.51446432] (0.9645884203858242+0j)
B2 24708.818786957007 (50, 'B2', 1, 'A1') [-264.91295521] (-0.9651734896745006+0j)
B2 24708.818908370573 (50, 'A1', 0, 'B2') [249.11429151] (-0.9651734896745006-0j)
[24708.81878741 24708.81890791]
[24708.81890005] [24708.81879579]
